In [1]:
import json, boto3, logging
from botocore.exceptions import ClientError

In [6]:
with open("/home/team3/repository/keys/API_KEYS.json", "r") as f:
    key = json.load(f)

# S3 버킷 정보 get
with open("/home/team3/repository/keys/DATA_SRC_INFO.json", "r") as f:
    bucket_info = json.load(f)

with open("/home/team3/repository/keys/GEMINI_API_KEY.json", "r") as f:
    gemini_api_key = json.load(f)

In [7]:
dynamo_table_name = bucket_info['restore_table_name']

In [8]:
dynamodb = boto3.resource(
        'dynamodb',
        aws_access_key_id=key['aws_access_key_id'],
        aws_secret_access_key=key['aws_secret_key'],
        region_name=key['region']
    )
table = dynamodb.Table(dynamo_table_name)

In [9]:
scan_kwargs = {}
data = []

In [6]:
# Pagination을 고려하여 모든 데이터를 가져오기
while True:
    response = table.scan(**scan_kwargs)
    data.extend(response.get('Items', []))
    scan_kwargs['ExclusiveStartKey'] = response.get('LastEvaluatedKey', None)
    if not scan_kwargs['ExclusiveStartKey']:
        break

In [15]:
wanted_data = [obj for obj in data if obj.get('site_symbol', '').upper() == "WAN"]
jobkorea_data = [obj for obj in data if obj.get('site_symbol', '').upper() == "JK"]
rocketpunch_data = [obj for obj in data if obj.get('site_symbol', '').upper() == "RP"]

In [8]:
str(wanted_data[5])

"{'company_id': Decimal('18028'), 'site_symbol': 'WAN', 'job_title': '정보보안 담당자', 'job_prefer': '정보보호 관련 학과 및 IT 관련학 전공 클라우드 보안솔루션 운영 경험이 있으신 분 인프라 취약점 점검 및 웹 모의해킹 수행 가능하신 분 ISO27701, ISMS-P 보안인증 대응 경험이 있으신 분 ISMS-P 인증심사원, CPPG, CISA, CISSP, 정보보안기사 자격증을 보유하신 분', 'crawl_url': 'https://www.wanted.co.kr/wd/230608', 'end_date': None, 'job_id': Decimal('230608'), 'crawl_domain': 'www.wanted.co.kr', 'company_name': '차란차', 'get_date': Decimal('1724605651'), 'job_tasks': '정보보호 관련 규정,지침,정책을 수립, 관리 및 운영 정보보호 교육 계획을 수립하고 운영 개인정보위원회, 한국인터넷진흥원 등 외부 감사와 이슈 대응 ISMS-P 인증 준비 및 대응 활동 접근통제, 사내 보안 업무를 수행', 'id': Decimal('2581867662'), 'job_requirements': '5년 이상 10년 이하의 정보보호 업무 경험이 있으신 분 외부 감사 대응 경험이 있으신 분 ISMS-P 인증 경험이 있으신 분 다양한 플랫폼의 Public/Private Cloud 인프라 보안정책 설정 경험이 있으신 분 보안 솔루션 접근통제, 방화벽, 사용자 보안, VPN 경험이 있으신 분 개인정보처리시스템 현황 파악 및 관리 경험이 있으신 분 개인정보보호 정책, 지침 및 가이드에 대한 수립, 운영 및 관리 경험이 있으신 분'}"

In [9]:
str(jobkorea_data[3])

"{'indurstry_type': '소프트웨어 자문,개발,공급/컴퓨터주변기기 도소매', 'site_symbol': 'JK', 'job_title': '광화문 흥국생명 차세대시스템 구축 Java, BXM 프레임웍, 융자업무 관련 경험자를 모집 합니다. (s01)', 'job_category': '응용 소프트웨어 개발 및 공급업', 'crawl_url': 'https://www.jobkorea.co.kr/Recruit/GI_Read/45423573', 'end_date': '1724598000', 'required_career': False, 'job_id': '45423573', 'company_name': '행복추구 패러다임㈜kcms', 'resume_required': True, 'start_date': '1724511600', 'get_date': Decimal('1724605790'), 'job_tasks': '광화문 흥국생명 차세대시스템 구축', 'id': Decimal('2352333735'), 'stacks': 'JAVA, BXM 프레임웍, 융자업무 관련 경험자'}"

In [16]:
rocketpunch_data[0]

{'indurstry_type': '플랫폼',
 'site_symbol': 'RP',
 'crawl_url': 'https://www.rocketpunch.com/jobs/151639',
 'start_date': Decimal('1724166000'),
 'get_date': Decimal('1724227352'),
 'job_tasks': '이런 업무를 함께해요 SNS 자동화 솔루션 모두그램 리뉴얼 프론트엔드 개발- SNS 기반 재택부업 서비스 뷰업 리뉴얼 프론트엔드 개발- 개발 중이거나 개발된 프로젝트의 코드 품질 개선 및 리팩토링',
 'required_career': False,
 'id': Decimal('1777632098'),
 'crawl_domain': 'www.rocketpunch.com',
 'job_requirements': '스마일드래곤의 핵심 서비스를 개발하는 프론트엔드 팀에 합류하실 분을 모집합니다 저희는 이런 환경에서 일해요 개발팀 내부적으로 Slack 을 통한 커뮤니케이션이 이루어집니다.- Pull Request 는 반드시 팀원들의 코드 리뷰를 받고 이루어집니다.- Notion 을 통해 프로젝트 진행상황 및 정보를 관리합니다.- Docker 를 통해 개발환경을 단일화합니다.- GitHub Actions 를 통해 프로젝트 빌드 및 배포 자동화가 진행됩니다.- 사내 Kubernetes Helm Argo CD 환경으로 배포가 이루어집니다. 저희는 이런 기술을 사용해요 Next.js, TypeScript, Emotion, Zustand, React Query- Yarn Berry, pnpm, Rollup.js, Jest, Cypress더욱 자세한 기술스택은 https //stackshare.io/companies/smiledragon 에서 확인하실 수 있어요 이런 분과 함께하고 싶어요 3년 이상의 프론트엔드 개발 경험 혹은 그에 상응하는 실력을 갖추신 분- HTML5, CSS, JavaScript, TypeScript 에 대한 기본 지

In [10]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai
"""

import google.generativeai as genai

#genai.configure(api_key=os.environ["GEMINI_API_KEY"])
genai.configure(api_key=gemini_api_key['GEMINI_API'])

# Create the model
generation_config = {
  "temperature": 0.7,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

chat_session = model.start_chat(
  history=[
  ]
)

/home/hadoop/workspace/FPT5/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
wanted_data[6]

{'company_id': Decimal('3260'),
 'site_symbol': 'WAN',
 'job_title': 'QA 엔지니어',
 'job_prefer': 'CSTE, ISTQB, CSTS 등 자격증 보유자 성능테스트, 자동화테스트 경험자 개발 경력 보유자 CI/CD에 대한 이해와 경험 우대 Jenkins 등',
 'crawl_url': 'https://www.wanted.co.kr/wd/19232',
 'end_date': None,
 'job_id': Decimal('19232'),
 'crawl_domain': 'www.wanted.co.kr',
 'company_name': '모노리스',
 'get_date': Decimal('1724605651'),
 'job_tasks': 'Mobile App, Mobile Web, Back-office, 예약,판매관리시스템 등 모노리스에서 운영하는 서비스의 전반적인 품질 관리 서비스를 구성하는 전장, 트랙 센서 등 IoT 기기들에 적용되는 소프트웨어 품질 관리 소프트웨어 요구사항 및 구성에 대한 리뷰 피드백 결함 보고, 추적 및 분석에 따른 이슈 관리 개발 프로세스에서 다양한 기술이나 기법을 적용 더 좋은 개발문화를 함께 만들기',
 'id': Decimal('434466414'),
 'job_requirements': '5년 이상의 QA 업무 경험 iOS/Android Mobile Application QA 경험자 컴퓨터 관련 학과 졸업 or 그에 상응하는 역량 보유 빠른 프로세스에 유연하게 대응이 가능하신 분'}

In [ ]:
# WANTED propmt

# tech_stack_list = ["programming languages", "frameworks", "databases", "cloud services", "devops tools", "version control tools", "ci cd tools", "testing tools", "package management tools", "api management tools"]
# tech_stack_list_str = ', '.join(tech_stack_list)
info_keys = ["job_title", "job_prefer", "job_task", "job_requirements"]
info_keys_str = ', '.join(info_keys)
input_data = str(wanted_data[6])
input_text = f"""[문맥]\n당신은 IT-Software 전문 도메인을 가진 데이터 전문가입니다. 주어진 json형태의 데이터에서 필요한 데이터를 추출하거나 요약하는 임무를 맡았습니다.
[Instruction]
Input Data에서 주어진 json 포맷의 텍스트 데이터에서 {info_keys_str} key의 value 값을 바탕으로 아래에 주어진 key 값들에 대한 value 값을 채워주세요. 단, 모를 경우 \"null\"을 넣어주세요.
- `dev_stack`: Input Data를 기반으로 유추하거나 추출한 개발에 사용되는 프로그램 기술의 모음입니다. 해당 key의 value는 항상 영단어로 이루어진 list여야 합니다.
- `job_requirements`: Input data를 기반으로 유추한 직무의 요구사항입니다. 결과는 한글로 된 열 개 단어 이하인 문장 또는 단어이며, list 포맷으로 제시됩니다.
- `job_prefer`: Input data를 기반으로 유추한 직무의 우대사항입니다. 결과는 한글로 된 열 개 단어 이하의 문장 또는 단어이며, list 포맷으로 제시됩니다.
- `job_category`: Input Data를 기반으로 유추한 직무(position) 분류 입니다. 결과는 영단어로 이루어진 list여야 합니다.
- `indurstry_type`: Input Data에서 유추한 직무의 산업 분류입니다. value 값은 영단어로 이루어진 list여야 합니다.
- `required_career`: Input Data에서 경력을 요구유무를 나타냅니다. 경력일 경우 `True` BOOLEAN값을, 신입일 경우 `False` BOOLEAN값을 넣어주세요.
[Input Data]
{input_data}"""
#input_text = f"[문맥]\n당신은 IT-Software 전문 도메인을 가진 데이터 전문가입니다. 주어진 json형태의 데이터에서 필요한 데이터를 추출하거나 요약하는 임무를 맡았습니다.\n[Instruction]\nInput Data에서 주어진 json 포맷의 텍스트 데이터에서 {info_keys_str} key의 value 값을 바탕으로 아래에 주어진 key 값들에 대한 value 값을 채워주세요. 단, 모를 경우 \"null\"을 넣어주세요.\n- `tech_stack`: Input Data를 기반으로 유추하거나 추출한 개발에 사용되는 기술의 모음입니다. 해당 key의 value는 항상 영단어로 이루어진 list여야 합니다.\n- `job_requirements`: Input data를 기반으로 유추한 직무의 요구사항입니다. 결과는 한글로 된 열 개 단어 이하인 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_prefer`: Input data를 기반으로 유추한 직무의 우대사항입니다. 결과는 한글로 된 열 개 단어 이하의 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_category`: Input Data를 기반으로 유추한 직무(position) 분류 입니다. 결과는 영단어로 이루어진 list여야 합니다.\n- `indurstry_type`: Input Data에서 유추한 직무의 산업 분류입니다. value 값은 영단어로 이루어진 list여야 합니다.\n- `required_career`: Input Data에서 경력을 요구유무를 나타냅니다. 경력일 경우 `True` BOOLEAN값을, 신입일 경우 `False` BOOLEAN값을 넣어주세요.\n[Input Data]\n{input_data}"
response = chat_session.send_message(input_text)

print(response.text)

In [ ]:
# jobkorea propmt

info_keys = ["indurstry_type", "job_title", "job_category", "job_tasks", "stacks"]
info_keys_str = ', '.join(info_keys)
input_data = str(jobkorea_data[3])
input_text = f"[문맥]\n당신은 IT-Software 전문 도메인을 가진 데이터 전문가입니다. 주어진 json형태의 데이터에서 필요한 데이터를 추출하거나 요약하는 임무를 맡았습니다.\n[Instruction]\nInput Data에서 주어진 json 데이터에서 {info_keys_str} key의 값을 바탕으로 아래에 주어진 key 값들에 대한 value 값을 채워주세요. 단, 모를 경우 \"null\"을 넣어주세요. \n- `tech_stack`: Input Data를 기반으로 유추하거나 추출한 개발에 사용되는 기술의 모음입니다. 결과는 항상 영단어로 이루어진 list여야 합니다.\n- `job_requirements`: Input data를 기반으로 유추한 직무의 요구사항입니다. 결과는 한글로 된 열 개 단어 이하인 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_prefer`: Input data를 기반으로 유추한 직무의 우대사항입니다. 결과는 한글로 된 열 개 단어 이하의 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_category`: Input Data를 기반으로 유추한 직무(position) 분류 입니다. 결과는 영단어로 이루어진 list여야 합니다.\n- `indurstry_type`: Input Data의 직무의 산업 분류입니다. 결과는 영단어로 이루어진 list여야 합니다.\n[Input Data]\n{input_data}"
response = chat_session.send_message(input_text)

print(response.text)

In [27]:
# ROCKET-PUNCH propmt

info_keys = ["indurstry_type", "job_tasks", "job_requirements", "stacks"]
info_keys_str = ', '.join(info_keys)
input_data = str(rocketpunch_data[10])
input_text = f"""
[문맥]
당신은 IT-Software 전문 도메인을 가진 데이터 전문가입니다. 주어진 json형태의 데이터에서 필요한 데이터를 추출하거나 요약하는 임무를 맡았습니다.
[Instruction]
Input Data에서 주어진 json 포맷의 텍스트 데이터에서 {info_keys_str} key의 value 값을 바탕으로 아래에 주어진 key 값들에 대한 value 값을 채워주세요. 단, 모를 경우 \"null\"을 넣어주세요.
- `dev_stack`: Input Data를 기반으로 유추하거나 추출한 개발에 사용되는 프로그램 기술의 모음입니다. 해당 key의 value는 항상 영단어로 이루어진 list여야 합니다.
- `job_requirements`: Input data를 기반으로 유추한 직무의 요구사항입니다. 결과는 한글로 된 열 개 단어 이하인 문장 또는 단어이며, list 포맷으로 제시됩니다.
- `job_prefer`: Input data를 기반으로 유추한 직무의 우대사항입니다. 결과는 한글로 된 열 개 단어 이하의 문장 또는 단어이며, list 포맷으로 제시됩니다.
- `job_category`: Input Data를 기반으로 유추한 직무(position) 분류 입니다. 결과는 영단어로 이루어진 list여야 합니다.
- `indurstry_type`: Input Data에서 유추한 직무의 산업 분류입니다. value 값은 영단어로 이루어진 list여야 합니다.
[Input Data]
{input_data}"""
#input_text = f"[문맥]\n당신은 IT-Software 전문 도메인을 가진 데이터 전문가입니다. 주어진 json형태의 데이터에서 필요한 데이터를 추출하거나 요약하는 임무를 맡았습니다.\n[Instruction]\nInput Data에서 주어진 json 포맷의 텍스트 데이터에서 {info_keys_str} key의 value 값을 바탕으로 아래에 주어진 key 값들에 대한 value 값을 채워주세요. 단, 모를 경우 \"null\"을 넣어주세요.\n- `tech_stack`: Input Data를 기반으로 유추하거나 추출한 개발에 사용되는 기술의 모음입니다. 해당 key의 value는 항상 영단어로 이루어진 list여야 합니다.\n- `job_requirements`: Input data를 기반으로 유추한 직무의 요구사항입니다. 결과는 한글로 된 열 개 단어 이하인 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_prefer`: Input data를 기반으로 유추한 직무의 우대사항입니다. 결과는 한글로 된 열 개 단어 이하의 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_category`: Input Data를 기반으로 유추한 직무(position) 분류 입니다. 결과는 영단어로 이루어진 list여야 합니다.\n- `indurstry_type`: Input Data에서 유추한 직무의 산업 분류입니다. value 값은 영단어로 이루어진 list여야 합니다.\n- `required_career`: Input Data에서 경력을 요구유무를 나타냅니다. 경력일 경우 `True` BOOLEAN값을, 신입일 경우 `False` BOOLEAN값을 넣어주세요.\n[Input Data]\n{input_data}"
response = chat_session.send_message(input_text)

print(response.text)

```json
{
  "dev_stack": [
    "Python",
    "Linux",
    "C++",
    "Docker",
    "OpenCV"
  ],
  "job_requirements": [
    "개발 경력 3년 이상",
    "B2B 애플리케이션 개발 및 배포 경험",
    "C/C++, Python 개발 가능",
    "AI 비전 프로젝트 배포 경험",
    "객체 지향 프로그래밍 가능",
    "팀워크 및 의사소통 능력",
    "스타트업 근무 경험",
    "책임감"
  ],
  "job_prefer": [
    "자동화 프로젝트 개발 및 양산 적용 경험",
    "비전 광학계 설계 경험",
    "산업용 로봇 or 협동 로봇 이해",
    "Linux 환경 개발 경험",
    "3D 측정, 검사 경험",
    "딥러닝 모델 개발 경험",
    "정량적 결과 도출 능력"
  ],
  "job_category": [
    "Software Engineer",
    "AI Engineer",
    "Robotics Engineer"
  ],
  "indurstry_type": [
    "Robotics",
    "AI",
    "Industrial Automation",
    "Software",
    "Technology"
  ]
}
```


In [25]:
rocketpunch_data[7]

{'indurstry_type': '헬스케어',
 'site_symbol': 'RP',
 'crawl_url': 'https://www.rocketpunch.com/jobs/119282',
 'start_date': Decimal('1714834800'),
 'get_date': Decimal('1724227362'),
 'job_tasks': '스프린트 팀은 누구나 꾸준히 건강 관리를 할 수 있게 도와준다는 미션 아래 꾸준히 할 수 있는 환경을 만들어주기 위해 고민하고 실험하고 있습니다.- 스프린트는 건강 관리, 그 중 불편했던 식단 기록을 혁신적으로 바꾸고 있는 스타트업입니다. 미친듯이 편리한 사용성으로 업계 평균 대비 높은 리텐션을 달성하고 있습니다.- 출시 후 마케팅 집행 없이 순수 유입과 사용자 간 추천만으로 꾸준하게 성장하고 있습니다. 좋은 제품을 만들어 초기 사용자를 만족시키고 만족한 사용자가 주변 사람을 데려오는 모델로 커가고 있습니다.- 비전을 같이 바라봐주는 든든한 투자자님들에게 성장성과 사업성을 인정받아 투자를 받았습니다. 또 여러 유명 기관투자자에게서 투자 제안을 받고 있습니다.- 꾸준한 성장과 탄탄한 재방문율을 바탕으로 대한민국 뿐만 아니라 하반기에는 전 세계 사용자에게 새로운 가치를 전달하려고 합니다.',
 'required_career': False,
 'id': Decimal('488899403'),
 'crawl_domain': 'www.rocketpunch.com',
 'job_requirements': '담당업무 안정적이고 높은 성능을 가진 모던 웹 애플리케이션 개발 Git 플로우를 통한 전체적인 소프트웨어 프로세스 관리 및 테스트 서비스 운영 모니터링 및 장애 대응자격요건 꾸준하게, 높은 품질을 가진 산출물을 약속한 시간 안에 만들 수 있는 팀 플레이어 현재 방식에 만족하지 않고, 생산성을 더 올릴 수 있는 방법을 끊임없이 고민하는 팀 플레이어 알맞은 때에, 좋은 커뮤니케이션 능력으로 피드백을 주고 받는 것을 즐기는 팀 플레

In [ ]:
# PROGRAMMERS prompt
info_keys = ["job_title", "job_prefer", "job_task", "job_requirements", "job_category", "job_requirements"]
info_keys_str = ', '.join(info_keys)
input_data = str(programmers_data[0])
input_text = f"""[문맥]\n당신은 IT-Software 전문 도메인을 가진 데이터 전문가입니다. 주어진 json형태의 데이터에서 필요한 데이터를 추출하거나 요약하는 임무를 맡았습니다.
[Instruction]
Input Data에서 주어진 json 포맷의 텍스트 데이터에서 {info_keys_str} key의 value 값을 바탕으로 아래에 주어진 key 값들에 대한 value 값을 채워주세요. 단, 모를 경우 \"null\"을 넣어주세요.
- `dev_stack`: Input Data를 기반으로 유추하거나 추출한 개발에 사용되는 프로그램 기술의 모음입니다. 해당 key의 value는 항상 영단어로 이루어진 list여야 합니다.
- `job_requirements`: Input data를 기반으로 유추한 직무의 요구사항입니다. 결과는 한글로 된 열 개 단어 이하인 문장 또는 단어이며, list 포맷으로 제시됩니다.
- `job_prefer`: Input data를 기반으로 유추한 직무의 우대사항입니다. 결과는 한글로 된 열 개 단어 이하의 문장 또는 단어이며, list 포맷으로 제시됩니다.
- `job_category`: Input Data를 기반으로 유추한 직무(position) 분류 입니다. 결과는 영단어로 이루어진 list여야 합니다.
- `indurstry_type`: Input Data에서 유추한 직무의 산업 분류입니다. value 값은 영단어로 이루어진 list여야 합니다.
[Input Data]
{input_data}"""
#input_text = f"[문맥]\n당신은 IT-Software 전문 도메인을 가진 데이터 전문가입니다. 주어진 json형태의 데이터에서 필요한 데이터를 추출하거나 요약하는 임무를 맡았습니다.\n[Instruction]\nInput Data에서 주어진 json 포맷의 텍스트 데이터에서 {info_keys_str} key의 value 값을 바탕으로 아래에 주어진 key 값들에 대한 value 값을 채워주세요. 단, 모를 경우 \"null\"을 넣어주세요.\n- `tech_stack`: Input Data를 기반으로 유추하거나 추출한 개발에 사용되는 기술의 모음입니다. 해당 key의 value는 항상 영단어로 이루어진 list여야 합니다.\n- `job_requirements`: Input data를 기반으로 유추한 직무의 요구사항입니다. 결과는 한글로 된 열 개 단어 이하인 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_prefer`: Input data를 기반으로 유추한 직무의 우대사항입니다. 결과는 한글로 된 열 개 단어 이하의 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_category`: Input Data를 기반으로 유추한 직무(position) 분류 입니다. 결과는 영단어로 이루어진 list여야 합니다.\n- `indurstry_type`: Input Data에서 유추한 직무의 산업 분류입니다. value 값은 영단어로 이루어진 list여야 합니다.\n- `required_career`: Input Data에서 경력을 요구유무를 나타냅니다. 경력일 경우 `True` BOOLEAN값을, 신입일 경우 `False` BOOLEAN값을 넣어주세요.\n[Input Data]\n{input_data}"
response = chat_session.send_message(input_text)

print(response.text)

```json
{
  "dev_stack": [
    "Java",
    "JavaScript",
    "Spring",
    "Oracle",
    "MSSQL",
    "AWS",
    "Azure",
    "Google Cloud"
  ],
  "job_requirements": [
    "IT 프로젝트 수행 경력 6년 이상",
    "PL 수행 경험 2년 이상",
    "웹 프로그래밍 능숙자",
    "RDBMS 활용 가능자",
    "고객 요구사항 분석 및 시스템 설계 가능자"
  ],
  "job_prefer": [
    "구매 관련 프로젝트 수행 경력",
    "엠로 개발 프레임워크 및 유사 SCM 솔루션 개발 경험",
    "클라우드 플랫폼 경험",
    "프로젝트 관리 자격증 보유"
  ],
  "job_category": [
    "Server",
    "Backend",
    "Frontend"
  ],
  "indurstry_type": [
    "IT",
    "Software",
    "SCM"
  ]
}
```


In [ ]:
programmers_data[0]

{'company_id': Decimal('1023'),
 'site_symbol': 'PRO',
 'job_title': '프로그램 개발 엔지니어(PM/PL)',
 'job_category': '서버/백엔드, 프론트엔드',
 'job_prefer': '구매 관련 프로젝트 수행 경력 보유자 엠로 개발프레임워크 및 유사 SCM 솔루션 개발 경험자 클라우드 플랫폼(AWS, Azure, Google Cloud) 경험 우대 프로젝트 관리 자격증(PMP) 보유자',
 'crawl_url': 'https://career.programmers.co.kr/api/job_positions/8394',
 'end_date': None,
 'status': 'released',
 'post_status': True,
 'required_career': True,
 'job_id': Decimal('8394'),
 'crawl_domain': 'https://career.programmers.co.kr/',
 'company_name': '엠로',
 'resume_required': True,
 'start_date': '2024-07-03',
 'get_date': Decimal('20240828'),
 'job_tasks': '공급망관리(SCM) 솔루션 설계 및 개발 구매업무 프로세스 최적화 및 시스템 구축 프로젝트 수행 고객 요구사항 분석 및 솔루션 제안',
 'id': Decimal('2902426399'),
 'stacks': 'Java, JavaScript, Spring, Oracle, MSSQL(Microsoft SQL Server)',
 'job_requirements': 'IT 프로젝트 수행 경력 6년 이상 (PL 수행경험 2년 이상 포함) 웹 프로그래밍 능숙자 (Java, JavaScript, Spring framework 등) RDBMS(Oracle, MS-SQL, MySQL, PostgreSQL 등) 활용가능자 고객 요구사항 분석 및 시스템 설계 가능자'}